In [ ]:
from colabfit.tools.database import MongoDatabase, load_data
from colabfit.tools.property_settings import PropertySettings

client = MongoDatabase('colabfit_database', nprocs=1)

In [ ]:
client.insert_property_definition('tag:staff@noreply.openkim.org,2014-04-15:property/bulk-modulus-isothermal-cubic-crystal-npt')

In [ ]:
client.configurations.distinct('elements', {})

In [ ]:
client.configurations.distinct('labels', {'nsites': {'$lt': 10}})

# Verifying dataset correctness

In [ ]:
client.datasets.distinct('name')

In [ ]:
ds_id = client.datasets.find_one({'name': 'Mo_PRM2019'})['_id']
dataset = client.get_dataset(ds_id)['dataset']
dataset.name, ds_id

In [ ]:
client.dataset_to_markdown(
    ds_id=ds_id,
    base_folder='/tmp',
    html_file_name='README',
    data_file_name=dataset.name+'.xyz',
    data_format='mongo',
    yscale='log'
)

In [ ]:
dataset = client.dataset_from_markdown(
    html_file_path='/tmp/README',
    verbose=True
)

In [ ]:
dataset['dataset'].name

In [ ]:
client.dataset_to_markdown(
    ds_id=dataset['_id'],
    base_folder='/tmp',
    html_file_name='README-2',
    data_file_name=dataset['dataset'].name+'-2.xyz',
    data_format='mongo',
    yscale='log'
)

In [ ]:
client.plot_histograms(dataset.aggregated_info['property_fields'], ids=dataset.property_ids)

# Number of configurations

In [ ]:
client.configurations.count_documents({})

# Number of properties

In [ ]:
client.properties.count_documents({})

# Number of datasets

In [ ]:
client.datasets.count_documents({})

# Get configurations

In [ ]:
configs = client.get_configurations('all', attach_properties=True, generator=False)
configs[0]

In [ ]:
configs[0].info

In [ ]:
"""
Get all Configurations

Get all properties that have a CO ID that's in the query
"""

cursor = client.properties.aggregate([
    {'$unwind': '$relationships.configurations'},
#     {'$match': {'relationships.configurations': {'$in': ids}}},
    {'$lookup': {
        'from': 'configurations',
        'localField': 'relationships.configurations',
        'foreignField': '_id',
        'as': 'linked_co'
    }}
])

next(cursor)

In [ ]:
len(list(cursor))

## Names of datasets

In [ ]:
sorted(list(client.datasets.find({}, {'name'})), key=lambda x: x['name'].lower())

# Number of configuration sets

In [ ]:
client.configuration_sets.count_documents({})

## All configuration sets, and their linked datasets

In [ ]:
cursor = client.configuration_sets.aggregate([
    {'$project': {'relationships.datasets': 1}},
    {'$unwind': '$relationships.datasets'},
    {'$project': {'ds_id': '$relationships.datasets'}},
    {'$lookup': {
        'from': 'datasets',
        'localField': 'ds_id',
        'foreignField': '_id',
        'as': 'linked_ds'
    }},
    {'$project': {'ds_name': '$linked_ds.name'}}
])

sorted(list(cursor), key=lambda x: x['ds_name'][0].lower())

## Configuration sets that are tied to more than one dataset

In [ ]:
client.configuration_sets.count_documents({'relationships.datasets.1': {'$exists': True}})

In [ ]:
cursor = client.configuration_sets.aggregate([
    {'$match': {'relationships.datasets.1': {'$exists': True}}},
    {'$project': {'relationships.datasets': 1}},
    {'$unwind': '$relationships.datasets'},
    {'$project': {'ds_id': '$relationships.datasets'}},
    {'$lookup': {
        'from': 'datasets',
        'localField': 'ds_id',
        'foreignField': '_id',
        'as': 'linked_ds'
    }},
    {'$project': {'ds_name': '$linked_ds.name'}}
])

list(cursor)

# Total number of atoms

In [ ]:
next(client.configurations.aggregate([
    {'$group': {'_id': None, 'sum': {'$sum': '$nsites'}}}
]))

# Total number of configuration labels

In [ ]:
cursor = client.configurations.aggregate([
#     { "$match": { "_id": { "$in": [1, 2] } } },
    {"$group": {"_id": None, "labels": { "$push": "$labels" }}},
    {
        "$project": {
            "labels": {
                "$reduce": {
                    "input": "$labels",
                    "initialValue": [],
                    "in": { "$setUnion": ["$$value", "$$this"] }
                }
            }
        }
    }
])

labels = next(cursor)['labels']
len(labels)

## Counts for each label

In [ ]:
cursor = client.configurations.aggregate([
    {'$unwind': '$labels'},
    {'$group': {'_id': '$labels', 'count': {'$sum': 1}}}
])

sorted(cursor, key=lambda x: x['count'], reverse=True)

# Property distributions

In [ ]:
client.properties.find_one({})

In [ ]:
client.properties.distinct('type')

In [ ]:
all_energies = client.get_data('properties', ['energy-forces-stress.energy', 'energy-forces-virial.energy'], ravel=True, verbose=True)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for k,v in all_energies.items():        
    _ = ax.hist(v, bins=100, label=k)
    
ax.legend()
ax.set_yscale('log')

In [ ]:
all_forces = client.get_data('properties', ['energy-forces-stress.forces', 'energy-forces-virial.forces'], ravel=True, verbose=True)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for k,v in all_forces.items():
    _ = ax.hist(v, bins=100, label=k)
    
ax.legend()
ax.set_yscale('log')

In [ ]:
all_stresses = client.get_data('properties', ['energy-forces-stress.stress', 'energy-forces-virial.virial'], ravel=True, verbose=True)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for k,v in all_stresses.items():
    _ = ax.hist(v, bins=100, label=k)
    
ax.legend()
ax.set_yscale('log')

In [ ]:
client.properties.distinct('energy-forces-stress.stress.source-unit')

In [ ]:
client.properties.distinct('energy-forces-virial.virial.source-unit')

In [ ]:
client.datasets.find_one({'name': 'WBe_PRB2019'})

In [ ]:
client.properties.find_one({'relationships.datasets': '-6889328512840717042'})